In [1]:
import pandas as pd
import random
import numpy as np
from utilities.api_functions import *

# load the actual lookup table with real GVV IDs
geoid_lu_df = pd.read_csv('/Users/joshpaul/epa-justice/repo/epa-justice/tbl/geoid_lookup.csv')
# or load a test lookup table with multiple areas per (bogus) GVV ID
# geoid_lu_df = pd.read_csv('/Users/joshpaul/epa-justice/repo/epa-justice/tbl/geoid_lookup_multi_test.csv')
# get a random GVV ID for testing
gvv_id = geoid_lu_df['id'].iloc[random.sample(range(len(geoid_lu_df['id'])), 1)].values[0]

In [2]:
dhc = fetch_census_data_and_compute("dhc", gvv_id, geoid_lu_df)
print(f"Records returned: {len(dhc)}")
print(dhc)

fetching data from: https://api.census.gov/data/2020/dec/dhc?get=P12_001N,P12_002N,P12_026N,P12_003N,P12_004N,P12_005N,P12_006N,P12_020N,P12_021N,P12_022N,P12_023N,P12_024N,P12_025N,P12_027N,P12_028N,P12_029N,P12_030N,P12_044N,P12_045N,P12_046N,P12_047N,P12_048N,P12_049N&for=place:40670&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
Records returned: 1
   GEOID    GEOID  total_population  pct_65_plus  pct_under_18
0    2.0  40670.0             792.0     14.89899     20.328283


In [3]:
acs5 = fetch_census_data_and_compute("acs5", gvv_id, geoid_lu_df)
print(f"Records returned: {len(acs5)}")
print(acs5)

fetching data from: https://api.census.gov/data/2020/acs/acs5/subject?get=S1810_C03_001E,S1810_C03_001M,S2701_C03_001E,S2701_C03_001M,S2701_C05_001E,S2701_C05_001M&for=place:40670&in=state:02&key=839fc96162a9e16e7896434e7592eccaf7938706
Records returned: 1
   pct_w_disability  moe_pct_w_disability  pct_insured  moe_pct_insured  \
0              16.9                   6.2         74.9              7.8   

   pct_uninsured  moe_pct_uninsured  GEOID    GEOID  
0           25.1                7.8    2.0  40670.0  


In [50]:
#areatype_str = "county"
#areatype_str = "place"
areatype_str = "zcta"

#geoid = "090"
#geoid = "24230"
geoid = "99709"

for var_str in var_dict["cdc"]["PLACES"]["vars"].keys():
    base_url = var_dict["cdc"]["PLACES"]["url"][areatype_str]
    if areatype_str in ["county", "place"]:
        full_geoid = str("02"+geoid)
        data_val_type_str = var_dict["cdc"]["PLACES"]["vars"][var_str]["data_value_type_id"]
        url = f"{base_url}?measureid={var_str}&datavaluetypeid={data_val_type_str}&locationid={full_geoid}"
    else:
        # do not add state fips code to ZCTAs
        # do not specify data value type id... only crude prevalence is available for ZCTAs!
        url = f"{base_url}?measureid={var_str}&locationid={geoid}"
    print(url)
    
    with requests.get(url) as r:
        if r.status_code != 200:
            #TODO: raise error
            print("No response, check your URL")
        else:
            r_json = r.json()

    print(r_json[0]['data_value'])

for var_str in var_dict["cdc"]["SDOH"]["vars"].keys():
    base_url = var_dict["cdc"]["SDOH"]["url"][areatype_str]
    if areatype_str in ["county", "place"]:
        full_geoid = str("02"+geoid)
        url = f"{base_url}?measureid={var_str}&locationid={full_geoid}"
    else:
        # do not add state fips code to ZCTAs
        url = f"{base_url}?measureid={var_str}&locationid={geoid}"
    print(url)

    with requests.get(url) as r:
        if r.status_code != 200:
            #TODO: raise error
            print("No response, check your URL")
        else:
            r_json = r.json()

    print(r_json[0]['data_value'])


https://data.cdc.gov/resource/qnzd-25i4.json?measureid=CASTHMA&locationid=99709
9.3
https://data.cdc.gov/resource/qnzd-25i4.json?measureid=COPD&locationid=99709
4.6
https://data.cdc.gov/resource/qnzd-25i4.json?measureid=CHD&locationid=99709
3.8
https://data.cdc.gov/resource/qnzd-25i4.json?measureid=STROKE&locationid=99709
2.1
https://data.cdc.gov/resource/qnzd-25i4.json?measureid=DIABETES&locationid=99709
6.7
https://data.cdc.gov/resource/qnzd-25i4.json?measureid=KIDNEY&locationid=99709
2.2
https://data.cdc.gov/resource/bumh-rgsq.json?measureid=REMNRTY&locationid=99709
31.9
https://data.cdc.gov/resource/bumh-rgsq.json?measureid=NOHSDP&locationid=99709
3.7
https://data.cdc.gov/resource/bumh-rgsq.json?measureid=POV150&locationid=99709
14.1
https://data.cdc.gov/resource/bumh-rgsq.json?measureid=BROAD&locationid=99709
8.5


In [45]:
zcta = pd.read_csv('/Users/joshpaul/epa-justice/etc/PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release_20240418.csv')
zcta = zcta[zcta['MeasureId'].isin(list(var_dict["cdc"]["PLACES"]["vars"].keys()))]
zcta["measure_type"] = zcta["MeasureId"] + "_" + zcta["DataValueTypeID"]
zcta["measure_type"].unique()

/var/folders/dz/1ccfqg_n5mg3fsrt8rwm7fd80000gn/T/ipykernel_41143/3363655737.py:1: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  zcta = pd.read_csv('/Users/joshpaul/epa-justice/etc/PLACES__Local_Data_for_Better_Health__ZCTA_Data_2023_release_20240418.csv')


array(['CASTHMA_CrdPrv', 'CHD_CrdPrv', 'COPD_CrdPrv', 'DIABETES_CrdPrv',
       'KIDNEY_CrdPrv', 'STROKE_CrdPrv'], dtype=object)

In [44]:
place = pd.read_csv('/Users/joshpaul/epa-justice/etc/PLACES__Local_Data_for_Better_Health__Place_Data_2023_release_20240418.csv')
place_ak = place[(place['StateAbbr']=='AK') & (place['MeasureId'].isin(list(var_dict["cdc"]["PLACES"]["vars"].keys())))]
place_ak["measure_type"] = place_ak["MeasureId"] + "_" + place_ak["DataValueTypeID"]
place_ak["measure_type"].unique()

/var/folders/dz/1ccfqg_n5mg3fsrt8rwm7fd80000gn/T/ipykernel_41143/2539198368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  place_ak["measure_type"] = place_ak["MeasureId"] + "_" + place_ak["DataValueTypeID"]


array(['CASTHMA_CrdPrv', 'STROKE_CrdPrv', 'DIABETES_CrdPrv',
       'STROKE_AgeAdjPrv', 'CHD_CrdPrv', 'COPD_CrdPrv',
       'CASTHMA_AgeAdjPrv', 'KIDNEY_CrdPrv', 'COPD_AgeAdjPrv',
       'DIABETES_AgeAdjPrv', 'CHD_AgeAdjPrv', 'KIDNEY_AgeAdjPrv'],
      dtype=object)

In [42]:
county = pd.read_csv('/Users/joshpaul/epa-justice/etc/PLACES__Local_Data_for_Better_Health__County_Data_2023_release.csv')
county_ak = county[(county['StateAbbr']=='AK') & (county['MeasureId'].isin(list(var_dict["cdc"]["PLACES"]["vars"].keys())))]
county_ak["measure_type"] = county_ak["MeasureId"] + "_" + county_ak["DataValueTypeID"]
county_ak["measure_type"].unique()

/var/folders/dz/1ccfqg_n5mg3fsrt8rwm7fd80000gn/T/ipykernel_41143/1845687230.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_ak["measure_type"] = county_ak["MeasureId"] + "_" + county_ak["DataValueTypeID"]


array(['STROKE_CrdPrv', 'DIABETES_CrdPrv', 'CHD_AgeAdjPrv',
       'KIDNEY_CrdPrv', 'STROKE_AgeAdjPrv', 'DIABETES_AgeAdjPrv',
       'COPD_AgeAdjPrv', 'CHD_CrdPrv', 'CASTHMA_CrdPrv',
       'CASTHMA_AgeAdjPrv', 'COPD_CrdPrv', 'KIDNEY_AgeAdjPrv'],
      dtype=object)